## Competición del titanic-kaggle

En líneas generales, la idea con la competición es construir un modelo básico de Machine Learnign capaz de predecir la propabilidad, en términos de unos y ceros, que tienen los distintos pasajeros del titanic de salvarse (cada pasajero está identificado de manera única).

## Se cuentan con dos archivos iniciales:
* `test.csv`
* `train.csv`

A partir de estos archivos, debemos generar un CSV que sea capaz responda la pregunta de la competición, utilizando la estructura antes dicha, que se resume en **`0` para los que no se salvarán y `1` para los que sí lo harán**.

# Información del grupo:
* Deivy Peña - 1099429.
* Juan Avila - 1100378.
* Vladimir González - 1098139.
* Leonel Rodríguez - 1098665.


# 1. Importando librerías

In [9]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 2. Cargando y visualizando la data de manera inicial

In [16]:
print("Data de entrenamiento:")
data = pd.read_csv("train.csv")
data.head()

Data inicial:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
print("Data de prueba:")
test = pd.read_csv("test.csv")
test.head()

Data de prueba inicial:


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# 3. Extrayendo los ids de los pasajeros y creando la función clean

Esta función tiene por propósito limpiar la data del conjunto de datos que es provisto, a los fines de tener información que sea lo suficientemente sustanciosa para hacer las predicciones correspondinetes. Esta función recibe por parámetro el conjunto de datos que se vaya a analizar; puntualmente, esta función suprime las columnas que no aportan información relevante al modelo, posteriormente devuelve la data limpia, lista para ser utilizada en el modelo.

In [2]:
PassengerId = test.PassengerId
def clean(data):
    data = data.drop(["Ticket", "PassengerId", "Name", "Cabin"], axis=1)
    return data

data = clean(data)
test = clean(test)

test.head(2) #Comprobamos que tenemos solo las columnas que utilizaremos

# 4. Preprocesar los datos

Esto es, esencialmente, colocar algunas columnas cuyo valor resulta más complejo para el entendimiento del modelo y así, obtener mejores resultados. Las columnas que se modificarán serán, concretamente cuándo embarcó (embarked) y el sexo (sex), estos valores están en letras (string) en la visualización de la data inicial (véase punto 2). Ahora, los resultados se visualizarán de una manera distinta realizando el preprocesamiento de la data. Para ello, simplemente utilizamos las funciones transform y fit_transform que intercambian los valores a    `unos o ceros`, más sencillo para el entendimiento del modelo. 

In [5]:
def preprocess(data):
    values = {"SibSp": data.SibSp.median(), "Parch": data.Parch.median(), "Fare": data.Fare.median(), "Age": data.Age.median(), "Embarked": 0}
    data["Sex"] = data.Sex.map({"male": 1, "female": 0})
    data["Embarked"] = data.Embarked.map({"S": 1, "C": 2, "Q": 3})
    data = data.fillna(value=values)

    return data


trainingData = preprocess(data)
testData = preprocess(test)
      
trainingData.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


# 5. Aplicando el modelo de entenamiento a la data preprocesada

En primer lugar, se guardan los valores de la columna 'sobrevivientes' en la variable `x` y se guardan las demás columnas en la variable `y`. Una vez realizado esto, se aplica el modelo haciendo uso del `train_test_split` importado de `sklearn` en el punto 1. Se define el tamaño de la prueba que se guardará para realizar las validaciones (conjunto de comparación), este es el parámetro `test_size` y una semilla aleatoria para partir siempre desde un mismo punto en la predicción de los resultados (`random state`).

In [7]:
y = trainingData["Survived"]
X = trainingData.drop("Survived", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=4)

# 6. Creando un modelo de regresión logísitca

En este apartado, se crea un modelo de regresión logísitca, ajustado a los datos de entrenamiento obtenidos en el código anterior, esto se utilizará en lo adelante para predecir la supervivencia de los pasajeros del Titanic.

In [8]:
clf = LogisticRegression(random_state=1, max_iter=1000).fit(X_train, y_train)

# 7. Realizando las predicciones
Se realizan las predicciones partiendo del modelo de regresión logística y se determina entonces la exactitud del modelo a partir de los valores reales de supervivencia, en comparación con los predichos. Los resultados de las predicciones rondan el 81% de exactitud, por lo cual, se puede afirmar que el modelo ha surtido efecto. 

In [10]:
predictions = clf.predict(X_val)
accuracy_score(y_val, predictions)

0.8100558659217877

# 8. Realiznado las predicciones con los valores de prueba

En este caso, se utiliza el modelo para respecto a las características de los pasajeros en el conjunto de prueba `test`. Es decir, estas son las predicciones del modelo tomando en cuenta el comportamiento de regresión logística de los pasajeros y, guardándolos en una variable que será la utilizada para crear el archivo `CSV` solicitado por la competencia.

In [14]:
submission_preds = clf.predict(testData)

# 9. Creando el dataframe y el archivo de respuesta

En este paso, simplemente se organizan los resultados creando un dataframe con dos columnas: el ID del pasajero y la predicción de su supervivencia, construyéndolo en la forma requerida por la competición y convirtiendo este dataframe a un archivo CSV con el nombre que se establece en la descripción de la competición.

In [12]:
# Creando el data frame
df = pd.DataFrame({"PassengerId": test_ids.values,
                   "Survived": submission_preds,
                  })

# Convirtiendo el dataframe a csv
df.to_csv("submission.csv", index=False)